# 01_data_processing.ipynb

# NFL Data Processing

- NOTE: actual_totals WILL have some blank values since it is only calculated if the game has already been completed at the time the script is run.
- NOTE: predicted_totals and prediction_correct columns are calculated in the next notebook.
- NOTE: For the purposes of this script's output being consistent with our report, the processed_data file wil not be appended to the nfl_total_latest file since that would change our report's results. This script simply exists to show how the data cleaning/transformation takes place.

This notebook processes the raw data files and creates a standardized dataset matching the format of nfl_totals_latest.csv.

## Input Files:
- raw_odds_api.csv: Raw betting odds data
- raw_scores_api.csv: Raw game scores data
- raw_team_stats.csv: Raw team statistics

## Output File:
- processed_nfl_totals.csv: Processed and standardized dataset

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime

### Load all raw data files

In [ ]:
def load_raw_data():
    odds_df = pd.read_csv('raw_odds_api.csv')
    scores_df = pd.read_csv('raw_scores_api.csv')
    team_stats_df = pd.read_csv('raw_team_stats.csv')
    
    return odds_df, scores_df, team_stats_df

### Process raw odds data into standardized format

In [ ]:
def process_odds_data(odds_df):
    
    # Keep only totals market data with "Over" lines
    odds_df = odds_df[odds_df['name'] == 'Over'].copy()
    
    # Standardize column names
    odds_df = odds_df.rename(columns={
        'name': 'type',
        'price': 'odds',
        'bookmakers.key': 'bookmaker_key',
        'bookmakers.last_update': 'bookmaker_last_update'
    })
    
    # Convert timestamps
    odds_df['commence_time'] = pd.to_datetime(odds_df['commence_time'])
    odds_df['bookmaker_last_update'] = pd.to_datetime(odds_df['bookmaker_last_update'])
    
    # Convert numeric fields
    odds_df['odds'] = pd.to_numeric(odds_df['odds'])
    odds_df['point'] = pd.to_numeric(odds_df['point'])
    
    # Select and order columns to match final format
    return odds_df[[
        'type', 
        'odds', 
        'point',
        'commence_time',
        'home_team',
        'away_team',
        'bookmaker_key',
        'bookmaker_last_update'
    ]]

### Process raw scores data to calculate actual totals

In [ ]:
def process_scores_data(scores_df):
    
    def calculate_total(row):
        if pd.isna(row['scores']) or not row['completed']:
            return np.nan
        scores = eval(row['scores'])  # Safely evaluate string representation of scores list
        return sum(float(score['score']) for score in scores)
    
    scores_df['actual_total'] = scores_df.apply(calculate_total, axis=1)
    scores_df['commence_time'] = pd.to_datetime(scores_df['commence_time'])
    
    return scores_df[['home_team', 'away_team', 'commence_time', 'actual_total']]

### Process raw team statistics and standardize team names

In [ ]:
def process_team_stats(team_stats_df):
    
    # Team name mapping dictionary
    team_name_map = {
        'Arizona': 'Arizona Cardinals',
        'Atlanta': 'Atlanta Falcons',
        'Baltimore': 'Baltimore Ravens',
        'Buffalo': 'Buffalo Bills',
        'Carolina': 'Carolina Panthers',
        'Chicago': 'Chicago Bears',
        'Cincinnati': 'Cincinnati Bengals',
        'Cleveland': 'Cleveland Browns',
        'Dallas': 'Dallas Cowboys',
        'Denver': 'Denver Broncos',
        'Detroit': 'Detroit Lions',
        'Green Bay': 'Green Bay Packers',
        'Houston': 'Houston Texans',
        'Indianapolis': 'Indianapolis Colts',
        'Jacksonville': 'Jacksonville Jaguars',
        'Kansas City': 'Kansas City Chiefs',
        'Las Vegas': 'Las Vegas Raiders',
        'LA Chargers': 'Los Angeles Chargers',
        'Los Angeles Chargers': 'Los Angeles Chargers',
        'LA Rams': 'Los Angeles Rams',
        'Los Angeles Rams': 'Los Angeles Rams',
        'Miami': 'Miami Dolphins',
        'Minnesota': 'Minnesota Vikings',
        'New England': 'New England Patriots',
        'New Orleans': 'New Orleans Saints',
        'NY Giants': 'New York Giants',
        'New York Giants': 'New York Giants',
        'NY Jets': 'New York Jets',
        'New York Jets': 'New York Jets',
        'Philadelphia': 'Philadelphia Eagles',
        'Pittsburgh': 'Pittsburgh Steelers',
        'San Francisco': 'San Francisco 49ers',
        'Seattle': 'Seattle Seahawks',
        'Tampa Bay': 'Tampa Bay Buccaneers',
        'Tennessee': 'Tennessee Titans',
        'Washington': 'Washington Commanders'
    }
    
    # Apply team name standardization
    team_stats_df['Team'] = team_stats_df['Team'].map(team_name_map)
    
    # Convert rank and stats to numeric
    team_stats_df['Offense_Rank'] = pd.to_numeric(team_stats_df['Offense_Rank'])
    team_stats_df['Defense_Rank'] = pd.to_numeric(team_stats_df['Defense_Rank'])
    team_stats_df['Points_For'] = pd.to_numeric(team_stats_df['Points_For'])
    team_stats_df['Points_Against'] = pd.to_numeric(team_stats_df['Points_Against'])
    
    return team_stats_df

### Merge all processed data and format to match nfl_totals_latest.csv

In [ ]:
def merge_and_format_data(odds_df, scores_df, team_stats_df):
    
    # Merge odds and scores
    df = pd.merge(
        odds_df,
        scores_df,
        on=['home_team', 'away_team', 'commence_time'],
        how='left'
    )
    
    # Add team stats
    for team_type in ['home', 'away']:
        df = pd.merge(
            df,
            team_stats_df,
            left_on=f'{team_type}_team',
            right_on='Team',
            how='left'
        )
        
        # Rename columns appropriately
        df = df.rename(columns={
            'Offense_Rank': f'{team_type}_offense_rank',
            'Defense_Rank': f'{team_type}_defense_rank',
            'Points_For': f'{team_type}_points_for',
            'Points_Against': f'{team_type}_points_against'
        })
    
    # Add week numbers
    df['week'] = df['commence_time'].apply(determine_nfl_week)
    
    # Calculate predictions
    df['predicted_total'] = None  # To be calculated in analysis notebook
    df['prediction_correct'] = None  # To be calculated in analysis notebook
    
    # Final column ordering to match nfl_totals_latest.csv
    columns = [
        'type', 'odds', 'point', 'commence_time', 'home_team', 'away_team',
        'bookmaker_key', 'bookmaker_last_update', 'home_offense_rank',
        'home_points_for', 'home_defense_rank', 'home_points_against',
        'away_offense_rank', 'away_points_for', 'away_defense_rank',
        'away_points_against', 'actual_total', 'week', 'predicted_total',
        'prediction_correct'
    ]
    
    return df[columns]

### Main processing pipeline

In [ ]:
def main():

    # Load raw data
    print("Loading raw data files...")
    odds_df, scores_df, team_stats_df = load_raw_data()
    
    # Process each dataset
    print("Processing odds data...")
    odds_df = process_odds_data(odds_df)
    
    print("Processing scores data...")
    scores_df = process_scores_data(scores_df)
    
    print("Processing team stats...")
    team_stats_df = process_team_stats(team_stats_df)
    
    # Merge and format
    print("Merging and formatting data...")
    final_df = merge_and_format_data(odds_df, scores_df, team_stats_df)
    
    # Save processed data
    print("Saving processed data...")
    final_df.to_csv('processed_nfl_totals.csv', index=False)
    print("Processing complete! Output saved to processed_nfl_totals.csv")
    
    return final_df

if __name__ == "__main__":
    processed_df = main()